### **그래프 최단 경로 알고리즘의 동적 계획법 기반 분석**

### **목차**

- **1. 서론**
    - 1.1. 분석 대상 알고리즘
    - 1.2. 분석의 목적

- **2. 동적 계획법과 최단 경로 문제**
    - 2.1. 동적 계획법의 적용 조건
    - 2.2. 최단 경로 문제의 DP적 상태 정의

- **3. 단일 출발점 최단 경로 알고리즘**
    - 3.1. 벨만-포드(Bellman-Ford)
        - A. DP 상태 및 점화식
        - B. Python 구현
    - 3.2. 다익스트라(Dijkstra)
        - A. DP 상태 및 점화식
        - B. Python 구현

- **4. 모든 쌍 최단 경로 알고리즘**
    - 4.1. 플로이드-워셜(Floyd-Warshall)
        - A. DP 상태 및 점화식
        - B. Python 구현

- **5. 성능 비교 실험**
    - 5.1. 실험 설계
    - 5.2. 실험 결과 및 분석

- **6. 결론**

- **7. 부록**
    - 7.1. 실험 코드

---

### **1. 서론**

#### **1.1. 분석 대상 알고리즘**
본 보고서는 그래프의 최단 경로를 탐색하는 다음 세 가지 알고리즘을 분석한다.
-   벨만-포드(Bellman-Ford)
-   다익스트라(Dijkstra)
-   플로이드-워셜(Floyd-Warshall)

#### **1.2. 분석의 목적**
위 알고리즘들은 각기 다른 조건 하에서 최단 경로 문제를 해결한다. 본 보고서의 목적은 이 알고리즘들의 핵심 로직을 동적 계획법(Dynamic Programming, DP)의 관점에서 재해석하여, 상태(State)와 상태 전이(State Transition)를 기준으로 각 알고리즘의 구조적 공통점과 차이점을 명확히 하는 것이다.

### **2. 동적 계획법과 최단 경로 문제**

#### **2.1. 동적 계획법의 적용 조건**
DP는 다음 두 가지 조건을 만족하는 문제에 적용된다.
1.  **최적 부분 구조 (Optimal Substructure)**: 문제의 최적해가 부분 문제의 최적해로부터 구성된다.
2.  **중복되는 부분 문제 (Overlapping Subproblems)**: 동일한 부분 문제가 반복적으로 발생한다.

최단 경로 문제는 이 두 조건을 만족한다. `s`에서 `t`까지의 최단 경로 상의 임의의 정점 `v`에 대해, `s`에서 `v`까지의 부분 경로 역시 최단 경로이므로 최적 부분 구조가 성립한다.

#### **2.2. 최단 경로 문제의 DP적 상태 정의**
최단 경로 문제에서 DP의 상태(State)는 일반적으로 `dist` 배열로 정의된다. `dist[v]`는 특정 조건 하에서 시작점에서 정점 `v`까지의 최단 거리를 의미한다. 각 알고리즘은 이 "특정 조건"을 어떻게 정의하고 확장하느냐에 따라 구분된다.

### **3. 단일 출발점 최단 경로 알고리즘**

#### **3.1. 벨만-포드(Bellman-Ford)**

음수 가중치 간선을 허용하는 단일 출발점 최단 경로 알고리즘이다.

**A. DP 상태 및 점화식**
-   **상태 정의**: `D[k][v]` = 시작점 `s`에서 정점 `v`까지, 최대 `k`개의 간선을 사용하여 도달하는 최단 경로의 가중치.
-   **상태 전이 점화식**:
    $$
    D[k][v] = \min(D[k-1][v], \quad \min_{u \in \text{in-neighbors}(v)} \{ D[k-1][u] + w(u, v) \})
    $$
    실제 구현에서는 2차원 배열 `D[k][v]` 대신 1차원 `dist[v]` 배열을 `|V|-1`번 반복 갱신하는 방식으로 최적화한다.

**B. Python 구현**
```python
def bellman_ford(graph, start):
    num_vertices = len(graph)
    dist = {v: float('inf') for v in graph}
    dist[start] = 0

    for _ in range(num_vertices - 1):
        for u in graph:
            for v, weight in graph[u].items():
                if dist[u] != float('inf') and dist[u] + weight < dist[v]:
                    dist[v] = dist[u] + weight

    for u in graph:
        for v, weight in graph[u].items():
            if dist[u] != float('inf') and dist[u] + weight < dist[v]:
                return "Negative cycle detected"
    return dist
```

#### **3.2. 다익스트라(Dijkstra)**

음수 가중치 간선이 없는 그래프에서 동작하는 단일 출발점 최단 경로 알고리즘이다.

**A. DP 상태 및 점화식**
-   **상태 정의**: `dist[v]` = 시작점 `s`에서 정점 `v`까지의 현재까지 발견된 최단 거리.
-   **상태 전이 점화식 (Relaxation)**: 최단 거리가 확정된 정점 `u`가 선택될 때, `u`의 이웃 `v`에 대해 다음 점화식이 적용된다.
    $$
    \text{dist}[v] = \min(\text{dist}[v], \quad \text{dist}[u] + w(u, v))
    $$
    다익스트라 알고리즘은 그리디(Greedy) 전략을 통해 매 단계에서 `dist` 값이 가장 작은 정점을 선택함으로써, 상태 전이의 순서를 최적화한다.

**B. Python 구현**
```python
import heapq

def dijkstra(graph, start):
    dist = {v: float('inf') for v in graph}
    dist[start] = 0
    pq = [(0, start)]

    while pq:
        d, u = heapq.heappop(pq)
        if d > dist[u]:
            continue
        for v, weight in graph[u].items():
            if dist[u] + weight < dist[v]:
                dist[v] = dist[u] + weight
                heapq.heappush(pq, (dist[v], v))
    return dist
```

### **4. 모든 쌍 최단 경로 알고리즘**

#### **4.1. 플로이드-워셜(Floyd-Warshall)**

모든 정점 쌍 간의 최단 경로를 계산하는 알고리즘이다.

**A. DP 상태 및 점화식**
-   **상태 정의**: `D[k][i][j]` = 정점 `i`에서 `j`까지, `{0, 1, ..., k-1}` 집합의 정점만을 경유지로 사용했을 때의 최단 거리.
-   **상태 전이 점화식**:
    $$
    D[k][i][j] = \min(D[k-1][i][j], \quad D[k-1][i][k] + D[k-1][k][j])
    $$
    구현 시에는 `k`에 대한 차원을 생략하고 2차원 `dist` 배열을 갱신하는 방식으로 최적화한다.

**B. Python 구현**
```python
def floyd_warshall(graph):
    num_vertices = len(graph)
    dist = [[float('inf')] * num_vertices for _ in range(num_vertices)]
    
    for i in range(num_vertices):
        dist[i][i] = 0
    for u in graph:
        for v, weight in graph[u].items():
            dist[u][v] = weight

    for k in range(num_vertices):
        for i in range(num_vertices):
            for j in range(num_vertices):
                dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
    return dist
```

### **5. 성능 비교 실험**

#### **5.1. 실험 설계**

**A. 목적**
본 실험의 목적은 다익스트라, 벨만-포드, 플로이드-워셜 알고리즘의 이론적 시간 복잡도가 실제 실행 시간에 어떻게 반영되는지 실증적으로 검증하는 것이다. 그래프의 크기(정점 수)와 밀도(간선 수) 변화에 따른 각 알고리즘의 성능 추이를 측정하고 비교한다.

**B. 실험 환경 및 도구**
-   **실행 환경**: Google Colab (Python 3)
-   **주요 라이브러리**:
    -   `networkx`: 그래프 생성 및 조작
    -   `time` (`perf_counter`): 실행 시간 측정
    -   `numpy`: 행렬 연산 (플로이드-워셜)
    -   `matplotlib`: 결과 데이터 시각화
    -   `IPython.display.HTML`: 결과 테이블 출력

**C. 테스트 데이터 생성**
-   **그래프 모델**: `networkx.gnm_random_graph`를 사용하여 주어진 정점(V)과 간선(E) 수를 갖는 무작위 그래프를 생성한다.
-   **그래프 크기 (V)**: 정점의 수를 `[25, 50, 100, 200]`로 설정하여 변화시킨다.
-   **그래프 밀도 (E)**:
    -   **희소 그래프 (Sparse Graph)**: 간선의 수를 $E = 4V$로 설정.
    -   **밀집 그래프 (Dense Graph)**: 간선의 수를 $E = V^2 / 8$로 설정.
-   **가중치**: 모든 간선의 가중치는 `[1, 100]` 범위의 무작위 정수로 할당한다. 본 실험에서는 음수 가중치는 제외한다.

**D. 측정 방법**
-   각 그래프 유형(희소/밀집)과 크기 조합에 대해 3개의 알고리즘을 실행한다.
-   각 실행은 5회 반복하여 평균 실행 시간을 계산함으로써, 일시적인 시스템 부하로 인한 오차를 최소화한다.
-   측정 시간은 순수 알고리즘 실행 시간만 포함하며, 그래프 생성 및 데이터 구조 변환 시간은 제외한다.

---

#### **5.2. 실험 결과 분석**
![recursive](../image/g1.png)  
![recursive](../image/g2.png)  

-   **희소 그래프**: 다익스트라 알고리즘이 벨만-포드보다 월등히 빠른 성능을 보인다. 이는 $O(E \log V)$가 $O(VE)$보다 훨씬 효율적이기 때문이다. 플로이드-워셜은 정점 수에만 의존하므로 가장 느리다.
-   **밀집 그래프**: 간선 수가 $V^2$에 가까워지면 $O(E \log V)$와 $O(VE)$의 차이가 줄어든다. 다익스트라가 여전히 가장 빠르지만, 벨만-포드와의 성능 격차는 희소 그래프에 비해 감소한다. 플로이드-워셜은 간선 수와 무관하게 $O(V^3)$의 성능을 유지한다.
-   **전반적 경향**: 측정된 실행 시간은 각 알고리즘의 이론적 시간 복잡도와 일치하는 증가율을 보인다. 이는 이론적 분석이 실제 성능 예측에 유효함을 입증한다.

### **6. 결론**
본 보고서는 세 가지 주요 최단 경로 알고리즘을 DP 관점에서 분석했다.
-   벨만-포드는 **간선의 최대 개수**를 상태 확장 기준으로 사용한다.
-   플로이드-워셜은 **경유 가능한 정점의 집합**을 상태 확장 기준으로 사용한다.
-   다익스트라는 **최단 거리가 확정된 정점의 집합**을 기준으로 상태를 갱신하며, 그리디 접근법으로 이 과정을 최적화한다.

결론적으로 세 알고리즘은 각기 다른 상태 정의와 확장 방식을 사용하지만, 모두 **최적 부분 구조**에 기반하여 점화식을 통해 최적해를 도출하는 동적 계획법 원리를 공유한다.

```python
import time
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import heapq
import random

# ===================================================================
# 1. 알고리즘 구현
# ===================================================================

def dijkstra(graph, start):
    dist = {v: float('inf') for v in graph}
    dist[start] = 0
    pq = [(0, start)]
    while pq:
        d, u = heapq.heappop(pq)
        if d > dist[u]: continue
        for v, weight in graph[u].items():
            if dist[u] + weight < dist[v]:
                dist[v] = dist[u] + weight
                heapq.heappush(pq, (dist[v], v))
    return dist

def bellman_ford(graph, start):
    num_vertices = len(graph)
    dist = {v: float('inf') for v in graph}
    dist[start] = 0
    for _ in range(num_vertices - 1):
        for u in graph:
            for v, weight in graph[u].items():
                if dist[u] != float('inf') and dist[u] + weight < dist[v]:
                    dist[v] = dist[u] + weight
    return dist

def floyd_warshall(graph, num_vertices):
    dist = [[float('inf')] * num_vertices for _ in range(num_vertices)]
    for i in range(num_vertices):
        dist[i][i] = 0
    for u in graph:
        for v, weight in graph[u].items():
            dist[u][v] = weight
    for k in range(num_vertices):
        for i in range(num_vertices):
            for j in range(num_vertices):
                dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
    return dist

# ===================================================================
# 2. 실험 설계 및 실행
# ===================================================================

VERTICES_LIST = [25, 50, 100, 200]
NUM_RUNS = 5
START_NODE = 0

results_log = []

def run_experiment(graph_type):
    print(f"--- Running Experiment for {graph_type.upper()} GRAPHS ---")
    for v_count in VERTICES_LIST:
        if graph_type == 'sparse':
            e_count = v_count * 4
        else:
            e_count = int(v_count**2 / 8)
            max_edges = v_count * (v_count - 1) // 2
            if e_count > max_edges: e_count = max_edges

        G_nx = nx.gnm_random_graph(v_count, e_count)
        for (u, v) in G_nx.edges():
            G_nx.edges[u,v]['weight'] = random.randint(1, 100)
        
        graph_dict = {i: {} for i in range(v_count)}
        for u, v, data in G_nx.edges(data=True):
            graph_dict[u][v] = data['weight']
            graph_dict[v][u] = data['weight']

        times = {'dijkstra': [], 'bellman_ford': [], 'floyd_warshall': []}
        for _ in range(NUM_RUNS):
            start_t = time.perf_counter(); dijkstra(graph_dict, START_NODE); times['dijkstra'].append(time.perf_counter() - start_t)
            start_t = time.perf_counter(); bellman_ford(graph_dict, START_NODE); times['bellman_ford'].append(time.perf_counter() - start_t)
            start_t = time.perf_counter(); floyd_warshall(graph_dict, v_count); times['floyd_warshall'].append(time.perf_counter() - start_t)

        avg_times = {alg: np.mean(t) for alg, t in times.items()}
        print(f"V={v_count}, E={e_count}: Dijkstra={avg_times['dijkstra']:.6f}s, Bellman-Ford={avg_times['bellman_ford']:.6f}s, Floyd-Warshall={avg_times['floyd_warshall']:.6f}s")
        results_log.append({'type': graph_type, 'V': v_count, 'E': e_count, **avg_times})

run_experiment('sparse')
run_experiment('dense')

# ===================================================================
# 3. 결과 시각화 (그래프 및 테이블)
# ===================================================================

def plot_results(results, graph_type):
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 6))
    
    data = [r for r in results if r['type'] == graph_type]
    v_vals = [r['V'] for r in data]
    dijkstra_times = [r['dijkstra'] for r in data]
    bellman_times = [r['bellman_ford'] for r in data]
    floyd_times = [r['floyd_warshall'] for r in data]
    
    ax.plot(v_vals, dijkstra_times, 'o-', label='Dijkstra (O(E log V))', color='green')
    ax.plot(v_vals, bellman_times, 's--', label='Bellman-Ford (O(VE))', color='blue')
    ax.plot(v_vals, floyd_times, '^:', label='Floyd-Warshall (O(V^3))', color='red')
    
    ax.set_xlabel('Number of Vertices (V)')
    ax.set_ylabel('Execution Time (seconds)')
    ax.set_title(f'Algorithm Performance on {graph_type.capitalize()} Graphs')
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, which="both", ls="--")
    plt.show()

plot_results(results_log, 'sparse')
plot_results(results_log, 'dense')

# HTML 테이블 생성 (텍스트 색상 수정)
html_string = """
<style>
    .styled-table { width: 95%; margin: 25px auto; border-collapse: collapse; font-family: sans-serif; box-shadow: 0 0 20px rgba(0, 0, 0, 0.1); }
    .styled-table thead tr { background-color: #333; color: #ffffff; text-align: center; }
    /* --- CSS Changes Start Here --- */
    .styled-table th, .styled-table td { 
        padding: 12px 15px; 
        text-align: center; 
        color: #000000; /* 모든 셀의 텍스트 색상을 검은색으로 설정 */
    }
    /* --- CSS Changes End Here --- */
    .styled-table tbody tr { border-bottom: 1px solid #dddddd; background-color: #f8f8f8; }
    .styled-table tbody tr:last-of-type { border-bottom: 2px solid #333; }
</style>
<h3>Execution Time Results</h3>
<table class="styled-table">
<thead>
    <tr><th>Graph Type</th><th>Vertices (V)</th><th>Edges (E)</th><th>Dijkstra</th><th>Bellman-Ford</th><th>Floyd-Warshall</th></tr>
</thead>
<tbody>
"""

for r in results_log:
    html_string += f"""
    <tr>
        <td>{r['type'].capitalize()}</td>
        <td>{r['V']}</td>
        <td>{r['E']}</td>
        <td>{r['dijkstra']:.6f}s</td>
        <td>{r['bellman_ford']:.6f}s</td>
        <td>{r['floyd_warshall']:.6f}s</td>
    </tr>
    """

html_string += "</tbody></table>"
display(HTML(html_string))
```
